In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, CategoricalCrossentropy

 import necessary libraries and modules such as NumPy, Pandas, TensorFlow, Keras, and specific Keras components needed for building and training neural networks.

In [17]:
train_data = pd.read_csv('C:\\Users\\d3vsh\\Downloads\\Rock Paper Scissors SXSW.v12i.tensorflow\\train\\_annotations.csv')
valid_data = pd.read_csv('C:\\Users\\d3vsh\\Downloads\\Rock Paper Scissors SXSW.v12i.tensorflow\\valid\\_annotations.csv')
test_data = pd.read_csv('C:\\Users\\d3vsh\\Downloads\\Rock Paper Scissors SXSW.v12i.tensorflow\\test\\_annotations.csv')


read CSV files containing data for training, validation, and testing from specified file paths into Pandas DataFrames.

In [18]:
image_width, image_height = 224, 224

Defines the dimensions (width and height) to which the input images will be resized during preprocessing.

In [19]:
class_labels = {'Rock': 0, 'Paper': 1, 'Scissors': 2}


train_data = train_data.dropna(subset=['label'])
valid_data = valid_data.dropna(subset=['label'])
test_data = test_data.dropna(subset=['label'])

Creates a dictionary to map class labels ('Rock', 'Paper', 'Scissors') to numerical labels (0, 1, 2) for classification.
python
Removes rows with missing 'label' values from the training, validation, and test datasets.

In [20]:
y_train_classification = [class_labels[label] for label in train_data['label']]
y_valid_classification = [class_labels[label] for label in valid_data['label']]
y_test_classification = [class_labels[label] for label in test_data['label']]


Converts the class labels in the data to their corresponding numerical labels using the class_labels dictionary.

In [21]:
def load_and_preprocess_data(data):
    images = []
    bounding_boxes = []

    for index, row in data.iterrows():
        image_path = os.path.join('C:\\Users\\d3vsh\\Downloads\\Rock Paper Scissors SXSW.v12i.tensorflow', str(row['filename']))
        if os.path.exists(image_path):
            image = keras.preprocessing.image.load_img(image_path, target_size=(image_width, image_height))
            image = keras.preprocessing.image.img_to_array(image)
            image /= 255.0
            images.append(image)

            bounding_box = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
            bounding_boxes.append(bounding_box)

    return np.array(images), np.array(bounding_boxes)

Defines a function load_and_preprocess_data that takes a DataFrame data as input and processes the images and bounding boxes associated with it.

In [22]:
X_train, y_train_bounding_box = load_and_preprocess_data(train_data)
X_valid, y_valid_bounding_box = load_and_preprocess_data(valid_data)
X_test, y_test_bounding_box = load_and_preprocess_data(test_data)


Calls the load_and_preprocess_data function to load and preprocess images and bounding boxes for training, validation, and test datasets

In [23]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y_classification, y_bounding_box, batch_size):
        self.X = X
        self.y_classification = y_classification
        self.y_bounding_box = y_bounding_box
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.X))

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_X = self.X[start:end]
        batch_classification = self.y_classification[start:end]
        batch_bounding_box = self.y_bounding_box[start:end]
        return batch_X, {'classification': batch_classification, 'bounding_box': batch_bounding_box}

Defines a custom data generator class DataGenerator that inherits from keras.utils.Sequence. This class is used to generate batches of data for training, where each batch contains both images and their corresponding classification and bounding box labels.

In [25]:
def custom_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    
    
    conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
    maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    
    flatten = Flatten()(maxpool2)
    dense1 = Dense(128, activation='relu')(flatten)

    #
    bounding_box_output = Dense(4, activation='linear', name='bounding_box')(dense1)

   
    classification_output = Dense(num_classes, activation='softmax', name='classification')(dense1)
   
    model = Model(inputs=input_layer, outputs=[classification_output, bounding_box_output])
    
    return model

Defines a function custom_model that constructs a custom neural network model using Keras. The model has two output branches, one for classification and another for bounding box regression.

In [26]:
input_shape = (image_width, image_height, 3)
num_classes = len(class_labels)
model = custom_model(input_shape, num_classes)

Specifies the input shape (image dimensions) and the number of classes and constructs the custom model using the custom_model function.

In [27]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'classification': CategoricalCrossentropy(), 'bounding_box': MeanSquaredError()},
              metrics={'classification': 'accuracy', 'bounding_box': 'mae'})

Compiles the model with the Adam optimizer, two different loss functions (CategoricalCrossentropy for classification and MeanSquaredError for bounding box regression), and metrics for monitoring model performance.

In [28]:
batch_size = 32
epochs = 10

efines batch size and the number of training epochs.

In [29]:
train_data_generator = DataGenerator(X_train, y_train_classification, y_train_bounding_box, batch_size)
valid_data_generator = DataGenerator(X_valid, y_valid_classification, y_valid_bounding_box, batch_size)
test_data_generator = DataGenerator(X_test, y_test_classification, y_test_bounding_box, batch_size)


Creates instances of the DataGenerator class for training, validation, and test data, which will be used to generate batches of data during training